In [1]:
import numpy as np
np.random.seed(20)

In [2]:
import tensorflow as tf
print(tf.__version__)

# Check if TensorFlow is built with CUDA (GPU support)
print("Built with CUDA: ", tf.test.is_built_with_cuda())

# Check if a GPU device is available
print("GPU Available: ", tf.test.is_gpu_available())

2024-04-30 17:23:10.161358: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-30 17:23:10.200454: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 17:23:10.779040: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.1
Built with CUDA:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available:  True


2024-04-30 17:23:11.960394: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:0 with 21146 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:19:00.0, compute capability: 8.6
2024-04-30 17:23:11.960931: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:1 with 22190 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2024-04-30 17:23:11.961344: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:2 with 22190 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:67:00.0, compute capability: 8.6
2024-04-30 17:23:11.961731: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:3 with 21472 MB memory:  -> device: 3, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [4]:
X_train, y_train = np.load('X_train.npy'), np.load('y_train.npy')
X_valid, y_valid = np.load('X_valid.npy'), np.load('y_valid.npy')
X_test, y_test = np.load('X_test.npy'), np.load('y_test.npy')

In [19]:
def model_creater(kernel_size=(3,3), optimizer='adam', loss='categorical_crossentropy') :
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size, activation='relu', input_shape=(125, 250, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size, activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [20]:
from sklearn.model_selection import KFold, GridSearchCV

model = model_creater()

param_grid = {
    'kernel_size':[(3,3), (5,5), (7,7)],
    'batch_size':[32, 64],
    'epochs':[5, 10, 15, 20],
    'optimizer':['adam', 'sgd'],
    'loss':['categorical_crossentropy', 'binary_crossentropy'],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=5)

In [21]:
grid.fit(X_train, y_train)

TypeError: Cannot clone object '<keras.src.engine.sequential.Sequential object at 0x7f18d073e760>' (type <class 'keras.src.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
print('final params:', grid.best_params_)
print('final score:', grid.best_score_)